In [26]:
import pathlib
import torch
import torchvision
import os
from PIL import Image
from tqdm import tqdm
from IPython.display import display # to display images
import sys
sys.path.append("../")

from helper_code.resnet_model import ResnetModel
from helper_code.mario_buttons_dataset import MarioEpisode,MarioButtonsDataset


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.manual_seed(17)


batch_size = 128
learning_rate = 1e-3
epochs = 7
    
group =3#3
use_color = False

preload=False


print(f"loading dataset preload:{preload}")

transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((256,256)),
        torchvision.transforms.ToTensor(),
        
        #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

#mario_epsode = MarioEpisode('../mario_dataset/Rafael_hwfrb471_e3_8-3_win/',group_frames=group,use_color=use_color,transform=transforms,preload=False)
mario_dataset = MarioButtonsDataset('C:\\Users\\lavi\Desktop\\deep_learning\\Mario_IDM_DL\\mario_dataset\\'
                                    ,group_frames=group,use_color=use_color,transform=transforms,preload=False)
def _extract_action(file_name):
        #print(pathlib.PurePath(file_name).name)
        action_number = pathlib.PurePath(file_name).name.split("_")[5][1:]
        action_tensor = torch.zeros(8)
        action_bin = str(bin(int(action_number)))[2:].zfill(8) #remove 0b in start
        #print(action_bin)
        for i in range(8):
            action_tensor[i] = int(action_bin[i])
        return action_tensor

tot_frames = 0
guess = torch.Tensor([0, 0, 0, 1, 0, 1, 0, 0])
tot_correct = 0
for ep in tqdm(os.listdir('../mario_dataset/')):
    if not os.path.isdir(os.path.join('../mario_dataset/',ep)):
         continue
    for frame in os.listdir(f'../mario_dataset/{ep}/'):
        tot_frames += 1 
        action = _extract_action(frame)
        tot_correct += (guess == action).sum().item()

print(f"accuracy:{tot_correct/(tot_frames*8)}")
# tot_action = torch.zeros(8,dtype=torch.float32)
# for ep in tqdm(os.listdir('../mario_dataset/')):
#     if not os.path.isdir(os.path.join('../mario_dataset/',ep)):
#          continue
#     for frame in os.listdir(f'../mario_dataset/{ep}/'):
#         tot_frames += 1
#         tot_action += _extract_action(frame)
        
# tot_action = torch.round(tot_action/tot_frames,decimals=2)
# print(f"mean action :{tot_action}")


# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)
# model = ResnetModel(group_size=group,use_color=use_color,use_pretrained=False).to(device)
# model.load_state_dict(torch.load('C:\\Users\\lavi\Desktop\\deep_learning\\Mario_IDM_DL\\models\\checkpoint_1_2500.pt',map_location=device))

# model.eval()
# with torch.no_grad():
#     for i in [0,10,100,1000]:
#         img,action = mario_dataset[0]
#         img = img.unsqueeze(0)
#         img = img.to(device)
#         pred = torch.sigmoid(model(img))
#         print(f"pred:{pred.cpu().numpy()},action:{action}")

cuda:0
loading dataset preload:False
total episodes:280


 12%|█▏        | 33/281 [00:13<02:46,  1.49it/s]